# 환경설정

In [ ]:
import torch
import gc

# 1. 모든 변수 참조 해제 시도
if 'model' in locals():
    del model
if 'inputs' in locals():
    del inputs

# 2. 파이썬 가비지 컬렉션 실행
gc.collect()

# 3. PyTorch 캐시 비우기
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

print("GPU 메모리가 정리되었습니다.")

IndentationError: unexpected indent (3545016517.py, line 4)

In [2]:
# 설치(import)
import os
import torch
import gc
from PIL import Image
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration, BitsAndBytesConfig


c:\ddddwoo\project\Oneness\AiMind-AiModels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# VARCO VISION 2.0

In [3]:
# [필수] 메모리 파편화 방지 및 관리 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def clean_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

clean_memory()

model_id = "NCSOFT/VARCO-VISION-2.0-1.7B-OCR"

# 1. 4비트 양자화 + CPU 오프로딩 준비
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# 2. 모델 로드 (low_cpu_mem_usage 추가)
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="sdpa",
    low_cpu_mem_usage=True # CPU 메모리 사용 최적화
)
processor = AutoProcessor.from_pretrained(model_id)

# 3. 이미지 크기 극단적 축소 (성공 여부 확인용)
image_path = "test.png"
image = Image.open(image_path).convert("RGB")

# 512나 768로 대폭 낮춥니다. (6GB에서는 1024도 벅찰 수 있습니다)
target_size = 512 
w, h = image.size
if max(w, h) > target_size:
    scale = target_size / max(w, h)
    image = image.resize((int(w * scale), int(h * scale)), resample=Image.LANCZOS)

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "<ocr>"},
        ],
    },
]

# 4. 입력 데이터 처리
inputs = processor.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device)

# 5. 추론 (더 쥐어짜기)
clean_memory()

with torch.inference_mode():
    generate_ids = model.generate(
        **inputs,
        max_new_tokens=256, # 토큰 수를 확 줄여서 메모리 압박 해소
        do_sample=False,
        use_cache=True
    )

# 6. 결과 출력
output = processor.decode(generate_ids[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
print("\n=== OCR 결과 ===")
print(output)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



=== OCR 결과 ===
2021년0.083, 0.071, 0.213, 0.123
8월0.229, 0.071, 0.303, 0.123
1일0.323, 0.071, 0.389, 0.123
일요일0.397, 0.071, 0.49, 0.127
날씨0.51, 0.071, 0.57, 0.117
일어난0.083, 0.157, 0.15, 0.185
시간:0.153, 0.157, 0.21, 0.185
8시0.233, 0.151, 0.323,


In [4]:
import re

# ... (이전 추론 코드는 동일, max_new_tokens만 1024로 다시 늘려주세요)

with torch.inference_mode():
    generate_ids = model.generate(
        **inputs,
        max_new_tokens=1024, # 좌표값이 길어서 넉넉하게 줘야 끝까지 나옵니다.
        do_sample=False,
        use_cache=True
    )

# 결과 디코딩
full_output = processor.decode(generate_ids[0][len(inputs.input_ids[0]):], skip_special_tokens=True)

# 정규표현식을 사용해 숫자 좌표(0.123, 0.456 등)를 제거하는 함수
def clean_ocr_text(text):
    # 소수점 숫자가 포함된 좌표 패턴을 찾아 제거합니다.
    cleaned = re.sub(r'\d+\.\d+,\s*\d+\.\d+,\s*\d+\.\d+,\s*\d+\.\d+', '', text)
    # 남은 숫자 조각들이나 불필요한 공백 정리
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

print("\n=== 정제된 OCR 결과 ===")
print(clean_ocr_text(full_output))

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



=== 정제된 OCR 결과 ===
2021년 8월 1일 일요일 날씨 일어난 시간: 8시 참드는 시간: 9시 제목: 한국의 여름 그림일기 공모전 한국 무나원 에서 한 국 여름을 주 재료 그림 일기 공모전을 한다. 자세한 내용 은 홍폐이지
